In [2]:
### 불러와야하는 함수 (prompt에도 pip install 필요)
# > pip install plotly

import sys
import urllib.request
from urllib.request import urlopen
import pandas as pd
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolsrcValidator
import folium
import json
import numpy as np
import os
import requests
from openpyxl.workbook import Workbook
import matplotlib.pyplot as plt
plt.rc('font', family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
from matplotlib import font_manager, rc
import platform
if platform.system() == "Windows" :
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc("font", family = font_name)

### 네이버 API 클라이언트 id, pw (발급받아야함)

client_id = "jup4Xlhan3w6Eni5SnYG"
client_secret = "SZhlghBkCi"

In [153]:
### URL을 받아서 카테고리 구분을 하기 때문에 카테고리 변경시마다 URL 변경 필요.

# 분야별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/categories"
# 분야내 기기별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/category/device"
# 분야내 성별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/category/gender"
# 분야내 연령별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/category/age"
# 분야내 키워드별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/category/keywords"
# 분야내 키워드 기기별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/device"
# 분야내 키워드 성별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/gender"
# 분야내 키워드 연령별 트렌드 조회 : url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/age"

### body 내 카테고리의 code도 카테고리 변경시마다 변경해야함 (이건 바뀔 수도 있으니 항상 확인 필요)

# 식품 : 50000006
# 건강식품 : 50000023
# 건강환/정 : 50001899
# 건강분말 : 50001900
# 비타민제 : 50001090
# 인삼 : 50001902
# 한방재료 : 50001091
# 꿀 : 50001905
# 영양제 : 50001092
# 건강즙/과일즙 : 50007000
# 홍삼 : 50012060
# 환자식/영양보충식 : 50011861

### 키워드는 키워드 이름별로 밖에 받을 수 없음.

In [3]:
### 성별 클릭율 비교
url = "https://openapi.naver.com/v1/datalab/shopping/category/gender"

### 성별 연령별도 추가하고 싶으면 주석 풀고 '}' 위치 옮기고 ',' 추가하면 됨
body = '{"startDate":"2018-01-01",'
body += '"endDate":"2018-12-31",'
body += '"timeUnit":"month",'
body += '"category":"50001090"}'
# body += '"age":["10", "20", "30"]}'

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode == 200):
    response_body = response.read()
    data = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

### API로 받아온 Json 파일을 불러와서 list 형태로 저장 (보통 period(기간), ratio(비율), group(성별, 나이별)으로 구분됨)
json_data = json.loads(data)
data_list = json_data["results"][0]["data"]

### 데이터 프레임 형태로 바꾸며, 필요시 pivot 테이블 활용해서 데이터 전처리(가공)
df = pd.DataFrame(data_list)
df_pivot = df.pivot(index='period', columns='group', values='ratio').reset_index()

### 데이터 Series 형태로 받아서 Figure에 넣기
date_sgmt = df_pivot['period']
male_data = df_pivot['m'] * -1
female_data = df_pivot['f']

fig = go.Figure()

fig.add_trace(go.Bar(x = male_data,
                    y = date_sgmt,
                    name = '남성',
                    orientation = 'h',
                    text = round(-1*male_data, 2)))
fig.add_trace(go.Bar(x = female_data,
                    y = date_sgmt,
                    name = '여성',
                    orientation = 'h',
                    text = round(female_data, 2)))

fig.update_layout(title = '18년_비타민제 클릭율 (성별전체)',
                  title_font_size = 22,
                  barmode = 'overlay',
                  bargap = 0.0,
                  bargroupgap = 0,
                  xaxis = dict(tickvals = [-100, -75, -50, -25, 0, 25, 50, 75, 100],
                             ticktext = ['100', '75', '50', '25', '0', '25', '50', '75', '100'],
                             title = '비율',
                             title_font_size = 14),
                  yaxis = dict(tickvals = ['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01'],
                             ticktext = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'],
                             title = '기간',
                             title_font_size = 14))

fig.show()

In [113]:
url = "https://openapi.naver.com/v1/datalab/shopping/categories"
body = '{"startDate":"2018-01-01",'
body += '"endDate":"2022-12-31",'
body += '"timeUnit":"date",'
body += '"category":[{"name":"식품","param":["50000006"], "name":"건강식품","param":["50000023"]}]}'

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode == 200):
    response_body = response.read()
    data = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)
    
json_data = json.loads(data)
data_list = json_data["results"][0]["data"]
df = pd.DataFrame(data_list)
a = 0
for i in df['period'] :
    df.loc[a, 'year'] = i[:4]
    df.loc[a, 'mmdd'] = i[5:]
    a += 1
    
date_sgmt = df['period']
year_sgmt = df['year']
month_sgmt = df['mmdd']
ratio_sgmt = df['ratio']
ratio18 = df[df['year']=='2018']['ratio']
ratio19 = df[df['year']=='2019']['ratio']
ratio20 = df[df['year']=='2020']['ratio']
ratio21 = df[df['year']=='2021']['ratio']
ratio22 = df[df['year']=='2022']['ratio']

fig = go.Figure()

fig.add_trace(go.Scatter(x = date_sgmt,
                         y = ratio18,
                         mode = 'lines',
                         name = '2018'))
fig.add_trace(go.Scatter(x = date_sgmt,
                         y = ratio19,
                         mode = 'lines',
                         name = '2019'))
fig.add_trace(go.Scatter(x = date_sgmt,
                         y = ratio20,
                         mode = 'lines',
                         name = '2020'))
fig.add_trace(go.Scatter(x = date_sgmt,
                         y = ratio21,
                         mode = 'lines',
                         name = '2021'))
fig.add_trace(go.Scatter(x = date_sgmt,
                         y = ratio22,
                         mode = 'lines',
                         name = '2022'))

fig.update_layout(title = '연도별 건강식품 클릭율(18-22)',
                  title_font_size = 22,
                  width = 1600,
                  height = 800,
                  xaxis = dict(tickvals = ['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01'],
                             ticktext = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'],
                             title = '기간',
                             title_font_size = 14))

fig.update_xaxes(rangeslider_visible = True)

fig.show()

In [140]:
url = "https://openapi.naver.com/v1/datalab/shopping/categories"
body = '{"startDate":"2018-01-01",'
body += '"endDate":"2022-12-31",'
body += '"timeUnit":"month",'
body += '"category":[{"name":"식품","param":["50000006"], "name":"건강식품","param":["50000023"]}]}'

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode == 200):
    response_body = response.read()
    data = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)
    
json_data = json.loads(data)
data_list = json_data["results"][0]["data"]
df = pd.DataFrame(data_list)
a = 0
for i in df['period'] :
    df.loc[a, 'year'] = i[:4]
    df.loc[a, 'mmdd'] = i[5:]
    a += 1
    
date_sgmt = df['period']
year_sgmt = df['year']
month_sgmt = df['mmdd']
ratio_sgmt = df['ratio']

fig = go.Figure(data = go.Heatmap(x = month_sgmt, y = year_sgmt, z = ratio_sgmt, text = ratio_sgmt))
fig.update_layout(title = '연도별 건강식품 클릭율(18-22)',
                  title_font_size = 22,
                  width = 1600,
                  height = 800,
                  xaxis = dict(tickvals = ['01-01', '02-01', '03-01', '04-01', '05-01', '06-01', '07-01', '08-01', '09-01', '10-01', '11-01', '12-01'],
                             ticktext = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'],
                             title = '기간',
                             title_font_size = 14))
fig.update_xaxes(showgrid = True,
                 ticks = 'outside',
                 tickson = 'boundaries',
                 ticklen = 20)
fig.update_yaxes(showgrid = True,
                 ticks = 'outside',
                 tickson = 'boundaries',
                 ticklen = 20)
fig.show()

In [150]:
df_pivot = pd.pivot_table(df, values='ratio', index='year', columns='mmdd')
df_pivot
fig = px.imshow(df_pivot, text_auto=True, width=1600, height=800)
fig.update_layout(title = '연도별 건강식품 클릭율 Heatmap(18-22)',
                  title_font_size = 26,
                  width = 1600,
                  height = 800,
                  xaxis = dict(tickvals = ['01-01', '02-01', '03-01', '04-01', '05-01', '06-01', '07-01', '08-01', '09-01', '10-01', '11-01', '12-01'],
                               ticktext = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'],
                               title = '기간',
                               title_font_size = 14),
                  yaxis = dict(title = '연도',
                               title_font_size = 14))
fig.show()

In [151]:
url = "https://openapi.naver.com/v1/datalab/shopping/category/age"

body = '{"startDate":"2018-01-01",'
body += '"endDate":"2018-12-31",'
body += '"timeUnit":"month",'
body += '"category":"50000023"}'

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode == 200):
    response_body = response.read()
    data = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)
    
json_data = json.loads(data)
data_list = json_data["results"][0]["data"]
df = pd.DataFrame(data_list)
df_pivot = df.pivot(index='period', columns='group', values='ratio').reset_index()


fig = px.bar(df, x = 'ratio', y = 'period', color = 'group', orientation = 'h')

fig.update_layout(title = '월별 연령별 건강식품 클릭율 (18년)',
                  title_font_size = 22,
                  bargap = 0.0,
                  bargroupgap = 0,
                  yaxis = dict(tickvals = ['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01'],
                             ticktext = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'],
                             title = '기간',
                             title_font_size = 14))

fig.show()

In [102]:
fig = px.pie(df, values = 'ratio', names = 'group', hover_data = ['ratio'], labels = {'group' : 'ratio'},
             color_discrete_sequence=px.colors.sequential.Viridis, hole=.3)
fig.update_traces(textinfo = 'percent+label', textfont_size = 14)
fig.show()